In [1]:
%pip install transformers[torch]
%pip install evaluate


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

import torch
import transformers
import torch.nn as nn
#from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer,AutoModelForTokenClassification

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

tqdm.pandas()

device = torch.device('cpu')
print(device)

cpu


In [3]:
ex_data = pd.read_excel('./shared/ZXC_Ismail/data_mail.xlsx', sheet_name="deti.mail")

data = pd.DataFrame(ex_data, columns=['URL', 'Title', 'Description', 'Article Text', 'Keywords'])

In [4]:
data.head()

,URL,Title,Description,Article Text,Keywords
0,https://deti.mail.ru/article/kostyumy-na-novyj...,Костюмы на Новый год для детей своими руками,Сделать новогодний наряд для сына или дочки св...,Костюм Снежинки В качестве основы для костюма ...,Дети 3-7 лет
1,https://deti.mail.ru/article/esli-rebenok-veri...,Как создают детское новогоднее шоу: интервью с...,"Режиссер, сценарист и главный новогодний волше...",Если ребенок верит в сказку хотя бы до десяти ...,Семья
2,https://deti.mail.ru/article/gid-po-detskim-no...,30 крутых детских новогодних подарков на любой...,"Думаете о том, что подарить ребенку на Новый г...",Подарки до 1 000 рублей Наборы для создания сн...,Семья
3,https://deti.mail.ru/article/chto-delat-esli-s...,"Что делать, если с ГВ не сложилось: 4 совета","Лучшая еда для младенца – это, конечно, мамино...",Чаще практикуйте контакт кожа к коже Период но...,1 месяц - 6 месяцев
4,https://deti.mail.ru/article/novyj-god-blizko-...,Новый год близко: как звезды с детьми наряжают...,Главный атрибут праздника уже появился в домах...,"На календаре почти середина декабря, и знамени...",Семья


In [5]:
pd.unique(data['Keywords'])

array(['Дети 3-7 лет', 'Семья', '1 месяц - 6 месяцев',
       'Дети старше 7 лет', '1 год - 3 года', 'Новорожденные', 'Роды',
       'Беременность', 'Планирование', '7 месяцев - 12 месяцев',
       'Советы опытных мам'], dtype=object)

In [6]:
text = data['Article Text'][0]

In [7]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import OpenAI
# from langchain_core.messages import HumanMessage, SystemMessage
# from langchain_openai import ChatOpenAI, AzureChatOpenAI

# base_url = "http://mistral.vkcloud.eazify.net:8000/v1"
# base_url = "http://hackllm.vkcloud.eazify.net:8000/v1"
# base_url = "http://shwars.vkcloud.eazify.net:8000/v1"

# chat = ChatOpenAI(api_key="<key>",
#                 model = "tgi",
#                 openai_api_base = base_url,
#                 temperature=0.2)

# instruct = """
# Прочитай приведённый ниже в тройных обратных кавычках отзыв и запиши основные идеи в формате JSON следующего вида:
# {{
#   "пересказ" : [],
#   "" : []
# }}
# Отзыв: ```{review}```"
# """

# chat_template = ChatPromptTemplate.from_messages(
#     [
#         ("system", "Ты ответственный и прилежный работник. Твоя задача искать основную смысловую нагрузгу теста"),
#         ("human", instruct),
#     ]
# )

# res = chat.invoke(chat_template.format_messages(review=text[:1024]))
# print(res.content)

In [8]:
data['Tags'] = pd.factorize(data['Keywords'])[ 0 ]
data = data.dropna()

In [9]:
data

,URL,Title,Description,Article Text,Keywords,Tags
0,https://deti.mail.ru/article/kostyumy-na-novyj...,Костюмы на Новый год для детей своими руками,Сделать новогодний наряд для сына или дочки св...,Костюм Снежинки В качестве основы для костюма ...,Дети 3-7 лет,0
1,https://deti.mail.ru/article/esli-rebenok-veri...,Как создают детское новогоднее шоу: интервью с...,"Режиссер, сценарист и главный новогодний волше...",Если ребенок верит в сказку хотя бы до десяти ...,Семья,1
2,https://deti.mail.ru/article/gid-po-detskim-no...,30 крутых детских новогодних подарков на любой...,"Думаете о том, что подарить ребенку на Новый г...",Подарки до 1 000 рублей Наборы для создания сн...,Семья,1
3,https://deti.mail.ru/article/chto-delat-esli-s...,"Что делать, если с ГВ не сложилось: 4 совета","Лучшая еда для младенца – это, конечно, мамино...",Чаще практикуйте контакт кожа к коже Период но...,1 месяц - 6 месяцев,2
4,https://deti.mail.ru/article/novyj-god-blizko-...,Новый год близко: как звезды с детьми наряжают...,Главный атрибут праздника уже появился в домах...,"На календаре почти середина декабря, и знамени...",Семья,1
...,...,...,...,...,...,...
5962,https://deti.mail.ru/article/uslysh/,"""Услышь себя"". Тест для родителей",Часто ли родители слышат самих себя? Какова сп...,От вас требуется лишь максимально честно отмет...,Дети 3-7 лет,0
5963,https://deti.mail.ru/article/disp/,Что такое диспансеризация и зачем она нужна?,Диспансеризация - это метод медицинского обслу...,Самое главное при диспансеризации во время бер...,Беременность,7
5964,https://deti.mail.ru/article/lor/,Консультация у отоларинголога в период беремен...,У будущих мам часто возникает вопрос - с какой...,В настоящее время у 65-70% обследованных берем...,Беременность,7
5965,https://deti.mail.ru/article/test__nelepicy__d...,"Тест ""Нелепицы"". Для детей 3-6 лет",Образ мышления дошкольников представлен образа...,Для тестирования вам понадобится заранее подго...,Дети 3-7 лет,0


In [10]:
from transformers import AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=9)

Some weights of the model checkpoint at google-bert/bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-c

In [11]:
train_text = data['Article Text'][:1000].astype('str').to_list()
train_labels = data['Tags'][:1000].to_list()
val_text = data['Article Text'][100:200].astype('str').to_list()
val_labels =  data['Tags'][100:200].to_list()
test_text = data['Article Text'][200:400].astype('str').to_list()
test_labels =  data['Tags'][200:400].to_list()


In [12]:
dataset = [{"label" : v1, "text" : v2} for v1,v2 in zip(train_labels, train_text)]
#

In [13]:
def tokenize_function(examples):

    return tokenizer(examples["text"][:1024], padding='max_length', truncation=True, max_length=512)

dataset = pd.Series(dataset)

tokenized_datasets = dataset.map(tokenize_function)

In [14]:
small_train_dataset = list(tokenized_datasets)

small_eval_dataset = list(tokenized_datasets)

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

2024-04-21 13:09:11.736693: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/datadisk/jupyter-romchik/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [16]:
import evaluate

metric = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

model.resize_token_embeddings(len(tokenizer))

Embedding(28996, 768, padding_idx=0)

In [20]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.engine.keras_tensor import KerasTensor
# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased")
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5))  # No loss argument!

model.fit(tokenized_dataset, train_labels)

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
cannot import name 'call_context' from 'tensorflow.python.keras.engine' (/opt/conda/lib/python3.9/site-packages/tensorflow/python/keras/engine/__init__.py)

In [ ]:
sdlghskjghdjkgvhk